# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
# Importando as Bibliotecas
import pandas as pd
import requests

### 1. Lendo os dados

In [2]:
# Lendo os dados
df = pd.read_csv('SINASC_RO_2019.csv')

In [3]:
df.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


#### 1.1 Verificando se há dados duplicados

In [4]:
# Número de registros dos dados
qtd_registros = df.shape[0]

# Número de dados duplicados 
qtd_registros_duplicados = (df.duplicated().shape[0]) - qtd_registros

print(f"No total há {qtd_registros} linhas neste dataset.")
print(f"Há {qtd_registros_duplicados} linhas de dados duplicados neste dataset.")

No total há 27028 linhas neste dataset.
Há 0 linhas de dados duplicados neste dataset.


### 2. Verificando se há valores ```missing```  nas variáveis

In [5]:
# Criando uma função para contar os valores ausentes
def contar_missings(df):
    lista = []

    # Iterar pelas colunas do DataFrame
    colunas = df.columns
    for col in colunas:
        # Verificar se a coluna possui valores ausentes
        if df[col].isna().any():
            # Contar os valores ausentes na coluna
            contagem = df[col].isna().sum()
            # Adicionar um dicionário com o nome da coluna e a contagem de valores ausentes à lista
            lista.append({'coluna': col, 'valores_missing': contagem})

    # Criar um novo DataFrame a partir da lista
    df_valores_missing = pd.DataFrame(lista)
    
    # Ordenando a coluna valores_missings     
    df_valores_missing.sort_values(by="valores_missing",ignore_index=True, ascending=False, inplace=True)
    return df_valores_missing

In [6]:
# Contando os valores ausentes
df_missings = contar_missings(df)
display(df_missings)

,coluna,valores_missing
0,DTRECORIGA,27028
1,CODANOMAL,26814
2,IDADEPAI,19421
3,SERIESCMAE,12710
4,DTULTMENST,10072
5,CODOCUPMAE,2907
6,MESPRENAT,2867
7,QTDFILMORT,2098
8,QTDPARTNOR,1879
9,QTDPARTCES,1791


Ao todo 45 variáveis possuem `valores ausentes`.

### 3. Filtrando as colunas e aplicando a contagem de valores `missings`

In [7]:
df_filtrado = df.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

In [8]:
# Contando o número de valores missing das colunas filtradas
df_filtrado_missing = contar_missings(df_filtrado)
display(df_filtrado_missing)

,coluna,valores_missing
0,QTDFILVIVO,1573
1,GESTACAO,1232
2,ESTCIVMAE,317
3,ESCMAE,312
4,APGAR5,103
5,GRAVIDEZ,79


Após o filtro das colunas, há 6 variáveis com `valores ausentes`.

### 4. Removendo os valores missings da coluna `Apgar5`

In [9]:
df_apgar5 = df_filtrado[df_filtrado['APGAR5'].notna()]

In [10]:
# Total de linha após remoção do valores Na da coluna APGAR5
print(f"O número total de linhas após remoção dos valores ausentes é de {df_apgar5.shape[0]}.")

O número total de linhas após remoção dos valores ausentes é de 26925.


In [11]:
# Contando o número de valores missing das colunas filtradas
valores_missing_apgar5 = df_apgar5['APGAR5'].isna().sum()

print(f"A coluna APGAR5 possui {valores_missing_apgar5} valores missings.")

A coluna APGAR5 possui 0 valores missings.


### 5. Alterando os valores missings das colunas `ESTCIVMAE` e `CONSULTAS` para 9.

In [12]:
# Cria um cópia do df
df_sem_missings = df_apgar5.copy()

In [13]:
# Preencher valores ausentes na coluna 'ESTCIVMAE' com 9
df_sem_missings.loc[:,'ESTCIVMAE'].fillna(9, inplace=True)
df_sem_missings.loc[:,'CONSULTAS'].fillna(9, inplace=True)

### 6. Substituindo os valores faltantes da quantitativa (QTDFILVIVO) por zero.

In [14]:
df_sem_missings.loc[:,'QTDFILVIVO'].fillna(0, inplace=True)

### 7. Análise exploratória das colunas categóricas que possuem valores ausentes.

In [15]:
# Contagam dos valores missings 
contar_missings(df_sem_missings)

,coluna,valores_missing
0,GESTACAO,1216
1,ESCMAE,310
2,GRAVIDEZ,76


In [16]:
# Informações estatísticas das colunas categóricas
df_sem_missings.select_dtypes('object').describe().transpose()

,count,unique,top,freq
ESCMAE,26615,5,8 a 11 anos,15581
GESTACAO,25709,6,37 a 41 semanas,22523
GRAVIDEZ,26849,3,Única,26356


#### 7.1 Coluna `GRAVIDEZ`

In [17]:
df_sem_missings['GRAVIDEZ'].value_counts()

Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

A coluna `GRAVIDEZ`possui três valores distintos, entre eles, o valor 'Ùnica' é o que possui maior frequência, totalizando **26.356** repetições. Devido a esse frequência, será utilizado esse valor para substituir os valores ausentes. 

In [18]:
df_sem_missings['GRAVIDEZ'].fillna('Única', inplace=True)

#### 7.2 Coluna `GESTACAO`

In [19]:
df_sem_missings['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

A coluna `GESTACAO` possui seis valores distintos, entre eles, o valor '37 a 41 semanas' é o que possui maior frequência, totalizando 22.523 repetições. Devido a esse frequência, será utilizado essa valor para substituir os valores ausentes.

#### 7.3 Coluna `ESCMAE`

In [20]:
# Contagem do valores únicos 
df_sem_missings['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

In [21]:
# Valores únicos da coluna
df_sem_missings['ESCMAE'].unique()

array(['8 a 11 anos', '12 anos ou mais', '1 a 3 anos', '4 a 7 anos', nan,
       'Nenhuma'], dtype=object)

De acordo com a estrutura do SINASC a variável `ESCMAE` se refere aos anos de estudos concluídos, sendo categorizado da seguinte maneira:
1. Nenhuma;
2. 1 a 3 anos;
3. 4 a 7 anos;
4. 8 a 11 anos;
5. 12 e mais;
9. Ignorado;


Com o entendimento que os valores ausentes podem ter sido ignorados, eles serão substituídos pelo valor categórico `Ignorado`, que mesmo sendo um valor válido, não foi utilizado para nenhuma paciente.

In [22]:
# Conversão dos valores ausentes para 'Ignorado'
df_sem_missings['ESCMAE'].fillna('Ignorado', inplace=True)

In [23]:
# Contagem dos valores únicos após conversão dos valores ausentes
df_sem_missings['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Ignorado             310
Nenhuma               40
Name: ESCMAE, dtype: int64

### 8. Criando a categorização da coluna `Apgar5`

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.

In [24]:
# Condições para categorias
apgar_normal = (df_sem_missings['APGAR5'] >= 8) & (df_sem_missings['APGAR5'] <= 10)    
apgar_leve = (df_sem_missings['APGAR5'] >= 6) & (df_sem_missings['APGAR5'] <= 7)
apgar_moderada = (df_sem_missings['APGAR5'] >= 4) & (df_sem_missings['APGAR5'] <= 5)    
apgar_severa = df_sem_missings['APGAR5'] <= 3

In [25]:
# Criando coluna APGARS_CAT    
df_sem_missings.loc[apgar_normal, 'APGAR5_CAT'] = 'normal'
df_sem_missings.loc[apgar_leve, 'APGAR5_CAT'] = 'asfixia leve'
df_sem_missings.loc[apgar_moderada, 'APGAR5_CAT'] = 'asfixia moderada'
df_sem_missings.loc[apgar_severa, 'APGAR5_CAT'] = 'asfixia severa'

In [26]:
# Verificando valores da coluna APGARS_CAT
df_sem_missings['APGAR5_CAT'].unique()

array(['normal', 'asfixia leve', 'asfixia severa', 'asfixia moderada'],
      dtype=object)

### 9. Alterando o nome das colunas

In [27]:
# Convertendo o nomes das colunas para minusculo 
df_sem_missings.columns = df_sem_missings.columns.str.lower()

In [29]:
# Alterando os nomes das colunas
df_sem_missings.columns = [
    'loc_nasc', 'idade_mae', 'est_civ_mae',
    'esc_mae', 'qtd_fil_vivo', 'gestacao', 
    'gravidez', 'consultas', 'apgar_5', 'apgar_5_cat']